#### Tasca M10 T01

#### Exercici 1. Realitza web scraping de dues de les tres pàgines web proposades utilitzant BeautifulSoup primer i Selenium després. 
- http://quotes.toscrape.com
- https://www.bolsamadrid.es
- www.wikipedia.es (fes alguna cerca primer i escrapeja algun contingut)

In [100]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

Començarem pel primer link, la web de cites, i primer realitzarem el web scraping amb la llibreria BeautifulSoup.

In [5]:
URL = "http://quotes.toscrape.com"
page = requests.get(URL)

In [8]:
soup = BeautifulSoup(page.content, "html.parser")

Quan inspeccionem le web veiem que les cites s'inclouen en els contenidors 'div' i en la class 'quote'.

In [9]:
cites = soup.find_all("div", class_="quote")

In [10]:
for cita in cites:
    print(cita, end="\n"*2)

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
<span>by <small class="author" itemprop="author">J.K. 

Ara extraiem la informació que ens interessa: la cita, el nom de l'autor i les etiquetes (tags).

In [11]:
for cita in cites:
    text = cita.find("span", class_="text").text
    author = cita.find("small", class_="author").text
    tag_html = cita.find_all("a", class_="tag")
    tag_list=[]
    for tag in tag_html:
        tag_list.append(tag.text)
    print(text, author, tag_list)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” Albert Einstein ['change', 'deep-thoughts', 'thinking', 'world']
“It is our choices, Harry, that show what we truly are, far more than our abilities.” J.K. Rowling ['abilities', 'choices']
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” Albert Einstein ['inspirational', 'life', 'live', 'miracle', 'miracles']
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” Jane Austen ['aliteracy', 'books', 'classic', 'humor']
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” Marilyn Monroe ['be-yourself', 'inspirational']
“Try not to become a man of success. Rather become a man of value.” Albert Einstein ['adulthood', 'success', 'value']
“It is better to be hated for what you are than to b

Un cop hem extret les dades, creem un df i incluim la informació:

In [14]:
df_cites = pd.DataFrame(columns=['text', 'author', 'tags'])

In [15]:
for cita in cites:
    text=cita.find("span", class_="text").text
    author=cita.find("small", class_="author").text
    tag_html=cita.find_all("a", class_="tag")
    tag_list=[]
    for tag in tag_html:
        tag_list.append(tag.text)
    df_cites = pd.concat([df_cites, pd.DataFrame([{'text': text, 'author':author, 'tags': tag_list}])], ignore_index=True)

In [16]:
df_cites.head()

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"


La web té 10 pàgines i 10 cites per pàgina. Per poder guardar la informació de totes les pàgines creem un bucle:

In [20]:
df_cites = pd.DataFrame(columns=['text', 'author', 'tags'])

In [21]:
for i in range(1,11):
    url = "https://quotes.toscrape.com/page/" + str(i) + "/"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    cites = soup.find_all("div", class_="quote")
    for cita in cites:
        text = cita.find("span", class_="text").text
        author = cita.find("small", class_="author").text
        tag_html = cita.find_all("a", class_="tag")
        tag_list=[]
        for tag in tag_html:
            tag_list.append(tag.text)
        df_cites = pd.concat([df_cites, pd.DataFrame([{'text': text, 'author':author, 'tags': tag_list}])], ignore_index=True)
    

Comprovem que s'han guardat els 100 registres en el dataframe.

In [22]:
df_cites.count()

text      100
author    100
tags      100
dtype: int64

In [23]:
df_cites.to_csv("bs_cites.csv")

Fem el mateix amb Selenium:

In [5]:
URL = "http://quotes.toscrape.com"

opts = Options()
opts.add_argument('-headless=new')
browser = Chrome(options=opts)
browser.get(URL)

In [101]:
cites = browser.find_elements(By.CLASS_NAME, 'quote')

De nou, extraiem la informació que ens interessa: la cita, el nom de l'autor i les etiquetes (tags).

In [102]:
for cita in cites:
    text = cita.find_element(By.CLASS_NAME, 'text').text
    author = cita.find_element(By.CLASS_NAME, 'author').text
    tag_html = cita.find_elements(By.CLASS_NAME, 'tags')
    tag_list=[]
    for tag in tag_html:
        tag_list.append(tag.text)
    print(text, author, tag_list)

“The truth." Dumbledore sighed. "It is a beautiful and terrible thing, and should therefore be treated with great caution.” J.K. Rowling ['Tags: truth']
“I'm the one that's got to die when it's time for me to die, so let me live my life the way I want to.” Jimi Hendrix ['Tags: death life']
“To die will be an awfully big adventure.” J.M. Barrie ['Tags: adventure love']
“It takes courage to grow up and become who you really are.” E.E. Cummings ['Tags: courage']
“But better to get hurt by the truth than comforted with a lie.” Khaled Hosseini ['Tags: life']
“You never really understand a person until you consider things from his point of view... Until you climb inside of his skin and walk around in it.” Harper Lee ['Tags: better-life-empathy']
“You have to write the book that wants to be written. And if the book will be too difficult for grown-ups, then you write it for children.” Madeleine L'Engle ['Tags: books children difficult grown-ups write writers writing']
“Never tell the truth to 

In [8]:
df_cites = pd.DataFrame(columns=['text', 'author', 'tags'])

Inserim les dades en el df, iterant per totes les pàgines de la web:

In [9]:
for i in range(1,11):
    url = "https://quotes.toscrape.com/page/" + str(i) + "/"
    browser.get(url)
    cites = browser.find_elements(By.CLASS_NAME, 'quote')
    for cita in cites:
        text=cita.find_element(By.CLASS_NAME, 'text').text
        author=cita.find_element(By.CLASS_NAME, 'author').text
        tag_html=cita.find_elements(By.CLASS_NAME, 'tags')
        tag_list=[]
        for tag in tag_html:
            tag_list.append(tag.text)
        df_cites = pd.concat([df_cites, pd.DataFrame([{'text': text, 'author':author, 'tags': tag_list}])], ignore_index=True)
browser.quit()

In [10]:
df_cites.head()

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,[Tags: change deep-thoughts thinking world]
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,[Tags: abilities choices]
2,“There are only two ways to live your life. On...,Albert Einstein,[Tags: inspirational life live miracle miracles]
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,[Tags: aliteracy books classic humor]
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,[Tags: be-yourself inspirational]


In [11]:
df_cites.to_csv('sel_cites.csv')

Seguidament, fem el web scraping d'una pàgina de Wikipedia sobre "Marcas de atletas españoles de todos los tiempos de maratón". Igual que en el cas anterior, primer ho farem amb la llibreria BeautifulSoup i després amb Selenium.

In [69]:
URL = "https://es.wikipedia.org/wiki/Marat%C3%B3n"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

In [70]:
tables = soup.find_all("table", class_="wikitable")

Ens interessa la segona taula (índex 1), la que fa referència a les 10 millors marques en categoria femenina:

In [73]:
marques_table = tables[1]
print(marques_table)

<table class="wikitable">
<tbody><tr>
<th>Ranking
</th>
<th>Tiempo
</th>
<th>Atleta
</th>
<th>País
</th>
<th>Fecha
</th>
<th>Lugar
</th></tr>
<tr>
<td>1
</td>
<td>2:11:53
</td>
<td><a href="/wiki/Tigst_Assefa" title="Tigst Assefa">Tigst Assefa</a>
</td>
<td><span style="display:none;">Etiopía</span><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a class="mw-file-description" href="/wiki/Archivo:Flag_of_Ethiopia.svg" title="Bandera de Etiopía"><img alt="Bandera de Etiopía" class="mw-file-element" data-file-height="600" data-file-width="1200" decoding="async" height="10" src="//upload.wikimedia.org/wikipedia/commons/thumb/7/71/Flag_of_Ethiopia.svg/20px-Flag_of_Ethiopia.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/7/71/Flag_of_Ethiopia.svg/30px-Flag_of_Ethiopia.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/7/71/Flag_of_Ethiopia.svg/40px-Flag_of_Ethiopia.svg.png 2x" width="20"/></a></span> </span><a href="/wiki/Etiop%C3%ADa" title="E

Els noms de les columnes estan als contenidors 'th':

In [74]:
headers= marques_table.findAll("th")
columns=[]
for header in headers:
    columns.append(header.text)
print(columns)

['Ranking\n', 'Tiempo\n', 'Atleta\n', 'País\n', 'Fecha\n', 'Lugar\n']


Eliminem el salt de línia.

In [75]:
columns = [elemento.replace('\n', '') for elemento in columns]
print(columns)

['Ranking', 'Tiempo', 'Atleta', 'País', 'Fecha', 'Lugar']


Ara que ja tenim el nom de les columnes, creem el df:

In [76]:
df_marques = pd.DataFrame(columns=columns)

Extraiem el contingut de les files dels contenidors 'tr':

In [77]:
rows = marques_table.findAll("tr")

Ens quedem amb totes les files a partir de la segona (índex 1), ja que la primera conté el nom de les columnes.

In [78]:
rows = rows[1:]

Incorporem les dades en el df que hem creat, iterant per totes les files de la taula.

In [80]:
for row in rows:
    cells = row.findAll("td")
    ranking = cells[0].text
    tiempo = cells[1].text
    atleta = cells[2].text
    pais = cells[3].text
    fecha = cells[4].text
    lugar =  cells[5].text
    df_marques = pd.concat([df_marques, pd.DataFrame([{'Ranking': ranking, 'Tiempo' : tiempo, 'Atleta' : atleta, 'País': pais, 'Fecha' : fecha, 'Lugar': lugar}])])

In [81]:
df_marques.head(5)

,Ranking,Tiempo,Atleta,País,Fecha,Lugar
0,1\n,2:11:53\n,Tigst Assefa\n,Etiopía Etiopía\n,24 de septiembre de 2023\n,Berlín\n
0,2\n,2:13:44\n,Sifan Hassan\n,Países Bajos Países Bajos\n,8 de octubre de 2023\n,Chicago\n
0,3\n,2:14:04\n,Brigid Kosgei\n,Kenia Kenia\n,13 de octubre de 2019\n,Chicago\n
0,4\n,2:14:18\n,Ruth Chepngetich\n,Kenia Kenia\n,9 de octubre de 2022\n,Chicago\n
0,5\n,2:14:58\n,Amane Beriso Shankule\n,Etiopía Etiopía\n,4 de diciembre de 2022\n,Valencia\n


Eliminem els salts de línia '\n':

In [85]:
columns = ['Ranking', 'Tiempo', 'Atleta', 'País', 'Fecha', 'Lugar']

for column in columns:
    df_marques[column] = df_marques[column].str.replace('\n', '')

In [86]:
df_marques.head(5)

,Ranking,Tiempo,Atleta,País,Fecha,Lugar
0,1,2:11:53,Tigst Assefa,Etiopía Etiopía,24 de septiembre de 2023,Berlín
0,2,2:13:44,Sifan Hassan,Países Bajos Países Bajos,8 de octubre de 2023,Chicago
0,3,2:14:04,Brigid Kosgei,Kenia Kenia,13 de octubre de 2019,Chicago
0,4,2:14:18,Ruth Chepngetich,Kenia Kenia,9 de octubre de 2022,Chicago
0,5,2:14:58,Amane Beriso Shankule,Etiopía Etiopía,4 de diciembre de 2022,Valencia


Guardem el df en un csv:

In [98]:
df_marques.to_csv('bs_marques.csv')

Per últim, fem el mateix amb Selenium:

In [23]:
URL = "https://es.wikipedia.org/wiki/Marat%C3%B3n"

opts = Options()
opts.add_argument('-headless=new')
browser = Chrome(options=opts)
browser.get(URL)

In [24]:
tables=browser.find_elements(By.CLASS_NAME, 'wikitable')

Extraiem la taula dels records de la categoria femenina:

In [25]:
marques_table = tables[1]

Identifiquem el nom de les columnes de la taula.

In [26]:
headers= marques_table.find_elements(By.TAG_NAME, "th")
columns=[]
for header in headers:
    columns.append(header.text)
print(columns)

['Ranking', 'Tiempo', 'Atleta', 'País', 'Fecha', 'Lugar']


Creem el df amb les columnes anteriors:

In [27]:
df_marques = pd.DataFrame(columns = columns)

Ara extraiem la informació de les files, sense comptar la primera ja que correspon al nom de les columnes.

In [28]:
rows = marques_table.find_elements(By.TAG_NAME, "tr")
rows = rows[1:]

In [29]:
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    ranking = cells[0].text
    tiempo = cells[1].text
    atleta = cells[2].text
    pais = cells[3].text
    fecha = cells[4].text
    lugar = cells[5].text
    df_marques = pd.concat([df_marques, pd.DataFrame([{'Ranking': ranking, 'Tiempo' : tiempo, 'Atleta' : atleta, 'País': pais, 'Fecha' : fecha, 'Lugar':lugar}])])
browser.quit()   

In [30]:
df_marques.head(5)

,Ranking,Tiempo,Atleta,País,Fecha,Lugar
0,1,2:11:53,Tigst Assefa,Etiopía,24 de septiembre de 2023,Berlín
0,2,2:13:44,Sifan Hassan,Países Bajos,8 de octubre de 2023,Chicago
0,3,2:14:04,Brigid Kosgei,Kenia,13 de octubre de 2019,Chicago
0,4,2:14:18,Ruth Chepngetich,Kenia,9 de octubre de 2022,Chicago
0,5,2:14:58,Amane Beriso Shankule,Etiopía,4 de diciembre de 2022,Valencia


Finalment ho guardem en un arxiu csv.

In [31]:
df_marques.to_csv('sel_marques.csv')

#### Exercici 2. Documenta en un Word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.

Veure el document 'Sobre el Dataset', en arxiu pdf.

#### Exercici 3. Tria una pàgina web que tu vulguis i realitza web scraping mitjançant la llibreria Selenium primer i Scrapy després. 

La pàgina web escollida és: https://es.wikipedia.org/wiki/Anexo:Marcas_del_tenis_individual_masculino_(Era_Abierta). Conté els títols d'èlit obtinguts pels tenistes del ATP.

Fem el web scraping amb Selenium:

In [85]:
URL = "https://es.wikipedia.org/wiki/Anexo:Marcas_del_tenis_individual_masculino_(Era_Abierta)"

opts = Options()
opts.add_argument('-headless=new')
browser = Chrome(options=opts)
browser.get(URL)

Busquem les taules de la web:

In [86]:
tables=browser.find_elements(By.CLASS_NAME, 'wikitable')

Extraiem la taula que ens interessa "Títulos de élite", que és la tretzena de la web (index 12):

In [87]:
titols_taula = tables[12]

Extraiem els noms de les columnes:

In [88]:
headers= titols_taula.find_elements(By.TAG_NAME, "th")
columns=[]
for header in headers:
    columns.append(header.text)
print(columns)

['', 'Tenista', 'Total', 'Grand Slam', 'Torneo de Maestros*', 'Masters 1000**', 'JJ.OO.***']


Creem el df i extraiem la informació de les files de la taula:

In [89]:
df_titols = pd.DataFrame(columns = columns)

In [90]:
df_titols

,,Tenista,Total,Grand Slam,Torneo de Maestros*,Masters 1000**,JJ.OO.***


In [91]:
rows = titols_taula.find_elements(By.TAG_NAME, "tr")
rows = rows[1:]

In [92]:
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    tenista = cells[1].text
    total = cells[2].text
    gs = cells[3].text
    tdm = cells[4].text
    m1000 = cells[5].text
    jjoo = cells[6].text
    df_titols = pd.concat([df_titols, pd.DataFrame([{'Tenista': tenista, 'Total' : total, 'Grand Slam' : gs, 'Torneo de Maestros*': tdm, 'Masters 1000**' : m1000, 'JJ.OO.***':jjoo}])])
browser.quit()  

In [93]:
df_titols.head(5)

,,Tenista,Total,Grand Slam,Torneo de Maestros*,Masters 1000**,JJ.OO.***
0,NaN,Novak Djokovic,71,24,7,40,0
0,NaN,Rafael Nadal,59,22,0,36,1
0,NaN,Roger Federer,54,20,6,28,0
0,NaN,Ivan Lendl,31,8,5,18,0
0,NaN,Pete Sampras,30,14,5,11,0


Eliminem la columna de NaNs.

In [96]:
df_titols = df_titols.dropna(axis=1)
df_titols.head(5)

,Tenista,Total,Grand Slam,Torneo de Maestros*,Masters 1000**,JJ.OO.***
0,Novak Djokovic,71,24,7,40,0
0,Rafael Nadal,59,22,0,36,1
0,Roger Federer,54,20,6,28,0
0,Ivan Lendl,31,8,5,18,0
0,Pete Sampras,30,14,5,11,0


In [97]:
df_titols.to_csv('sel_titols.csv')